### Working with GLORYS Data

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import pandas as pd
import re # Import for regex
import shapefile
import geopandas as gpd
from matplotlib import gridspec

In [2]:
import matplotlib.path as mplPath## testing

In [4]:
ds = xr.open_dataset('/Users/sherine_aldrin/Downloads/CoOL/GLORYS/mercatorglorys12v1_gl12_mean_201609.nc')

In [5]:
ds

<xarray.Dataset> Size: 15GB
Dimensions:    (longitude: 4320, latitude: 2041, depth: 50, time: 1)
Coordinates:
  * longitude  (longitude) float32 17kB -180.0 -179.9 -179.8 ... 179.8 179.9
  * latitude   (latitude) float32 8kB -80.0 -79.92 -79.83 ... 89.83 89.92 90.0
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * time       (time) datetime64[ns] 8B 2016-09-16
Data variables:
    mlotst     (time, latitude, longitude) float64 71MB ...
    zos        (time, latitude, longitude) float64 71MB ...
    bottomT    (time, latitude, longitude) float64 71MB ...
    sithick    (time, latitude, longitude) float64 71MB ...
    siconc     (time, latitude, longitude) float64 71MB ...
    usi        (time, latitude, longitude) float64 71MB ...
    vsi        (time, latitude, longitude) float64 71MB ...
    thetao     (time, depth, latitude, longitude) float64 4GB ...
    so         (time, depth, latitude, longitude) float64 4GB ...
    uo         (time, depth, latitude, longitude) float64 4GB ...
    vo         (time, depth, latitude, longitude) float64 4GB ...
Attributes: (12/14)
    title:                         Monthly mean fields for product GLOBAL_REA...
    references:                    http://marine.copernicus.eu
    credit:                        E.U. Copernicus Marine Service Information...
    licence:                       http://marine.copernicus.eu/services-portf...
    contact:                       servicedesk.cmems@mercator-ocean.eu
    producer:                      CMEMS - Global Monitoring and Forecasting ...
    ...                            ...
    area:                          GLOBAL
    product:                       GLOBAL_REANALYSIS_001_030
    dataset:                       global-reanalysis-001-030-monthly
    source:                        MERCATOR GLORYS12V1
    product_user_manual:           http://marine.copernicus.eu/documents/PUM/...
    quality_information_document:  http://marine.copernicus.eu/documents/QUID...

In [6]:
def get_all_nc_files(folder):
    nc_files = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".nc"):
                nc_files.append(os.path.join(root, file))
                #nc_files.append(file)
    return nc_files

#### Getting the data

In [7]:
omg_ncs = get_all_nc_files('/Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_data')
omg_data = omg_ncs

In [28]:
glorys_base_path = "/Users/sherine_aldrin/Downloads/CoOL/GLORYS"
glorys_data = [os.path.join(glorys_base_path, file) for file in ['mercatorglorys12v1_gl12_mean_201609.nc',
                                                             'mercatorglorys12v1_gl12_mean_201610.nc',
                                                             'mercatorglorys12v1_gl12_mean_201710.nc'
                                                            ]]

### with all this data, now I want to actually put it into a summary csv file to plot later on

### NOW looping over all the temp and salinity files

In [30]:
### read glorysdata for each variable
def read_glorys_temperature(nc_path):
        ds = xr.open_dataset(nc_path, engine='netcdf4')
        temp = ds['thetao'].values
        depth = ds['depth'].values
        lat = ds['latitude'].values
        lon = ds['longitude'].values
        ds.close()
        return temp, depth, lat, lon

def read_glorys_salt(nc_path):
        ds = xr.open_dataset(nc_path, engine='netcdf4')
        depth = ds['depth'].values
        lat = ds['latitude'].values
        lon = ds['longitude'].values
        salt = ds['so'].values
        ds.close()
        return salt, depth, lat, lon

# returns distances between my point and all the other points, from Prof
def great_circle_distance(lon_ref, lat_ref, Lon, Lat):
    earth_radius = 6371000
    lon_ref_radians = np.radians(lon_ref)
    lat_ref_radians = np.radians(lat_ref)
    lons_radians = np.radians(Lon)
    lats_radians = np.radians(Lat)
    lat_diff = lats_radians - lat_ref_radians
    lon_diff = lons_radians - lon_ref_radians
    d = np.sin(lat_diff * 0.5) ** 2 + np.cos(lat_ref_radians) * np.cos(lats_radians) * np.sin(lon_diff * 0.5) ** 2
    h = 2 * earth_radius * np.arcsin(np.sqrt(d))
    return(h)

# get year from OMG filename
def extract_year_from_omg(fname):
    match = re.search(r'_(\d{4})\d{2}', fname)
    return match.group(1) if match else None

# get year from GLORYS filename
def extract_year_month_from_glorys(fname):
    match = re.search(r'(\d{6})(?=\.nc)', fname)
    if match:
        date_str = match.group(1)
        return date_str[:4], date_str[4:]  # year, month
    else:
        return None, None

In [34]:
# To have for the csv file
temp_summary_rows = []
salt_summary_rows = []

glorys_data = [f for f in glorys_data if os.path.isfile(f) and f.endswith(".nc")]
# Read GLORYS temperature
for glorys_file in glorys_data:
    print(f"Processing: {glorys_file}")
    year, month = extract_year_month_from_glorys(os.path.basename(glorys_file))

    temp, depth, lat_grid, lon_grid = read_glorys_temperature(glorys_file)
    salt, _, _, _ = read_glorys_salt(glorys_file)
    if temp is None or salt is None:
        raise RuntimeError("Failed to read GLORYS file")

    for path in omg_data:
        try:
            omg = xr.open_dataset(path)
            lat_omg = omg.attrs["latitude"]
            lon_omg = omg.attrs["longitude"]
            omg_year = str(omg.attrs.get("year", ""))
            omg_month = str(omg.attrs.get("month", "")).zfill(2)
        except Exception as e:
            print(f"Skipping {path} due to error: {e}")
            continue

        if omg_year != year or omg_month != month:
            continue

        # Find closest grid point
        Lon2D, Lat2D = np.meshgrid(lon_grid, lat_grid)
        dists = great_circle_distance(lon_omg, lat_omg, Lon2D, Lat2D)
        min_dist = dists.min() / 1000
        y, x = np.unravel_index(np.argmin(dists), dists.shape)

        temp_profile = temp[0, :, y, x]
        salt_profile = salt[0, :, y, x]

        if np.any(~np.isnan(temp_profile)) and np.any(temp_profile != 0):
            temp_summary_rows.append({
                "CTD_file": os.path.basename(path),
                "Var_type": "Theta",
                "GLORYS lon(X)": Lon2D[y, x],
                "GLORYS lat(Y)": Lat2D[y, x],
                "OMG lon(X)": lon_omg,
                "OMG lat(Y)": lat_omg,
                "Distance": min_dist,
                "Year": year,
                "Month": month,
                "Profile": temp_profile.tolist()
            })

        if np.any(~np.isnan(salt_profile)) and np.any(salt_profile != 0):
            salt_summary_rows.append({
                "CTD_file": os.path.basename(path),
                "Var_type": "Salinity",
                "GLORYS lon(X)": Lon2D[y, x],
                "GLORYS lat(Y)": Lat2D[y, x],
                "OMG lon(X)": lon_omg,
                "OMG lat(Y)": lat_omg,
                "Distance": min_dist,
                "Year": year,
                "Month": month,
                "Profile": salt_profile.tolist()
            })

# Save to CSV
with open("Temp_GLORYS_Profiles.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["CTD_file", "Var_type", "GLORYS lon(X)", "GLORYS lat(Y)", "OMG lon(X)", "OMG lat(Y)", "Distance", "Year", "Month", "Profile"])
    writer.writeheader()
    writer.writerows(temp_summary_rows)

print("Saved: ALL GLORYS_Temp_Profiles.csv")

with open("Salt_GLORYS_Profiles.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["CTD_file", "Var_type", "GLORYS lon(X)", "GLORYS lat(Y)", "OMG lon(X)", "OMG lat(Y)", "Distance", "Year", "Month", "Profile"])
    writer.writeheader()
    writer.writerows(salt_summary_rows)

print("Saved: ALL GLORYS_Salt_Profiles.csv")

Processing: /Users/sherine_aldrin/Downloads/CoOL/GLORYS/mercatorglorys12v1_gl12_mean_201609.nc
Processing: /Users/sherine_aldrin/Downloads/CoOL/GLORYS/mercatorglorys12v1_gl12_mean_201610.nc
Processing: /Users/sherine_aldrin/Downloads/CoOL/GLORYS/mercatorglorys12v1_gl12_mean_201710.nc
Saved: ALL GLORYS_Temp_Profiles.csv
Saved: ALL GLORYS_Salt_Profiles.csv


### Now with the csv's I need to sort by region too

In [35]:
omg_loc_path = '/Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_CTD_Locations_2016_2017_with_region.csv'
df = pd.read_csv(omg_loc_path)
df.head()

,File_ID,Type,Year,Month,Day,Hour,Minute,Second,Longitude,Latitude,Region
0,20160707_213510,CTDs,2016,7,7,21,35,10,-66.973570,73.902115,NW
1,20160709_034633,CTDs,2016,7,9,3,46,33,-68.658730,75.207280,NW
2,20160709_082456,CTDs,2016,7,9,8,24,56,-69.085870,75.420880,NW
3,20160709_125524,CTDs,2016,7,9,12,55,24,-69.704834,75.644600,NW
4,20160711_235711,CTDs,2016,7,11,23,57,11,-69.625770,77.477295,NW


### Now that I can isolate the omg points in their regions, I need to iterate through the csv, check the lon and lat, and see which region they're in

In [37]:
temps_csv = pd.read_csv('/Users/sherine_aldrin/Downloads/CoOL/GLORYS/Temp_GLORYS_Profiles.csv')
salt_csv = pd.read_csv('/Users/sherine_aldrin/Downloads/CoOL/GLORYS/Salt_GLORYS_Profiles.csv')
temps_csv.head()

,CTD_file,Var_type,GLORYS lon(X),GLORYS lat(Y),OMG lon(X),OMG lat(Y),Distance,Year,Month,Profile
0,CTD_20160916_140334.nc,Theta,-65.250000,74.750000,-65.246246,74.732773,1.918555,2016,9,"[3.53926207870245, 3.5407269671559334, 3.54145..."
1,CTD_20160923_153439.nc,Theta,-64.750000,81.250000,-64.780159,81.222504,3.099313,2016,9,"[-1.6127506121993065, -1.5929746180772781, -1...."
2,CTD_20160914_145226.nc,Theta,-52.416668,69.166664,-52.412819,69.198280,3.518933,2016,9,"[5.956328026950359, 5.970976911485195, 6.00613..."
3,CTD_20160924_155359.nc,Theta,-61.250000,73.750000,-61.269279,73.738861,1.376355,2016,9,"[4.7375408336520195, 4.736808389425278, 4.7368..."
4,CTD_20160928_133513.nc,Theta,-58.250000,74.000000,-58.261341,74.033386,3.728566,2016,9,"[5.022461637854576, 5.022461637854576, 5.02319..."


### Actually make the histogram

In [41]:
# Paths
glorys_temp_csv = "Temp_GLORYS_Profiles.csv"
glorys_salt_csv = "Salt_GLORYS_Profiles.csv"
ctd_base_path = "/Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_data"
omg_loc_path = '/Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_CTD_Locations.csv'
shapefile_path = '/Users/sherine_aldrin/Downloads/CoOL/GLORYS/sample_polygons/sample_polygons'
region_output = '/Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_CTD_Locations_2016_2017_with_region.csv'
output_dir = "glorys_histograms"
os.makedirs(output_dir, exist_ok=True)

# Filter CTD Locations to 2016 & 2017 and Assign Region
df = pd.read_csv(omg_loc_path)
df = df[df['Year'].isin([2016, 2017])].copy()

sf = shapefile.Reader(shapefile_path)
polygons = {sf.records()[s][0]: np.array(shape.points) for s, shape in enumerate(sf.shapes())}

region_labels = []
for lon, lat in zip(df['Longitude'], df['Latitude']):
    assigned = False
    for region, poly in polygons.items():
        path = mplPath.Path(poly)
        if path.contains_point((lon, lat)):
            region_labels.append(region)
            assigned = True
            break
    if not assigned:
        region_labels.append("Unassigned")

df['Region'] = region_labels
df.to_csv(region_output, index=False)
print("Saved:", region_output)

# Load Region Info
region_df = pd.read_csv(region_output)

# Helper Functions
def read_ctd_profile(path, var_type):
    try:
        ds = xr.open_dataset(path)
        if var_type == "Theta":
            values = ds["potential_temperature"].values
        elif var_type == "Salinity":
            values = ds["practical_salinity"].values
        else:
            return None, None
        depths = ds["depth"].values
        ds.close()
        return depths, values
    except:
        return None, None

def clean_profile(profile_string):
    if pd.isna(profile_string):
        return []
    nums = re.findall(r'[-+]?[\d]*\.?[\d]+', profile_string)
    return [float(x) for x in nums if float(x) != 0.0]

def load_glorys_data(csv_path, var_type):
    df = pd.read_csv(csv_path)
    df["Var_type"] = var_type
    df["Profile"] = df["Profile"].apply(clean_profile)
    return df

# Load GLORYS Summary Data
df_temp = load_glorys_data(glorys_temp_csv, "Theta")
df_salt = load_glorys_data(glorys_salt_csv, "Salinity")
df_all = pd.concat([df_temp, df_salt], ignore_index=True)
df_all["File_ID"] = df_all["CTD_file"].str.replace("CTD_", "", regex=False).str.replace(".nc", "", regex=False)
df_all = df_all.merge(region_df[["File_ID", "Region"]], on="File_ID", how="left")
df_all["Region"] = df_all["Region"].fillna("Unassigned")

# Compute Differences
diffs_by_group = {}
for (region, var_type), group in df_all.groupby(["Region", "Var_type"]):
    key = (region, var_type)
    region_diffs = []

    for _, row in group.iterrows():
        ctd_path = os.path.join(ctd_base_path, row["CTD_file"])
        if not os.path.exists(ctd_path):
            continue

        glorys_profile = np.array(row["Profile"])
        ctd_depths, ctd_values = read_ctd_profile(ctd_path, var_type)

        if (
            glorys_profile is None or len(glorys_profile) == 0 or
            ctd_values is None or len(ctd_values) == 0 or
            ctd_depths is None or len(ctd_depths) == 0
        ):
            continue

        interp_len = min(len(glorys_profile), len(ctd_values))
        common_depths = np.linspace(ctd_depths.min(), ctd_depths.max(), interp_len)
        ctd_interp = np.interp(common_depths, ctd_depths, ctd_values)
        glorys_interp = np.interp(common_depths,
                                np.linspace(ctd_depths.min(), ctd_depths.max(), len(glorys_profile)),
                                glorys_profile)

        region_diffs.extend(glorys_interp - ctd_interp)

    diffs_by_group[key] = region_diffs

# Plotting with GridSpec
region_positions = {
    "N": (0, 0), "NW": (1, 0), "CW": (2, 0), "SW": (3, 0),
    "NE": (0, 1), "CE": (1, 1), "SE": (2, 1)
}
regions = list(region_positions.keys())

for var_type in ["Theta", "Salinity"]:
    fig = plt.figure(figsize=(20, 10))
    fig.suptitle(f"{var_type} Differences Across Regions (GLORYS - CTD)", fontsize=16)
    gs = gridspec.GridSpec(4, 2, figure=fig)

    for region in regions:
        row, col = region_positions[region]
        ax = fig.add_subplot(gs[row, col])
        key = (region, var_type)
        diffs = diffs_by_group.get(key, [])

        if not diffs:
            ax.set_title(f"{region} (No Data)")
            ax.axis("off")
            continue

        diffs = np.array(diffs)
        diffs = diffs[~np.isnan(diffs)]
        diffs = diffs[np.abs(diffs) < 5]  # remove extreme outliers

        if len(diffs) == 0:
            ax.set_title(f"{region} (No Valid Data)")
            ax.axis("off")
            continue

        mean_diff = np.mean(diffs)
        ax.hist(diffs, bins=50, color="red", edgecolor="black")
        ax.axvline(mean_diff, color="skyblue", linestyle="--", label=f"Mean = {mean_diff:.2f}")
        ax.set_title(region)
        ax.set_xlabel("GLORYS - CTD")
        ax.set_ylabel("Count")
        ax.grid(True)
        ax.legend()

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    filename = f"GLORYS_{var_type}_hist_by_region.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()

print(f"Saved plots to: {output_dir}")


Saved: /Users/sherine_aldrin/Downloads/CoOL/GLORYS/OMG_CTD_Locations_2016_2017_with_region.csv
Saved plots to: glorys_histograms
